In [2]:
pip install redis

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [1]:
################################   final code   #################################

from pymongo import MongoClient
import sys
from datetime import timedelta
import datetime
from pprint import pprint
from datetime import datetime
import redis
import json
import pandas as pd
import numpy as np
import time
import threading
from time import sleep
import xlwings as xw
#import AttachExpiry
##############################################################################################################
#variables required= index spot, index futures, mid strike, current expiry , far expiry
from pymongo import MongoClient
from datetime import datetime

def get_expiry_names(symbol, instrument):
    myClient = MongoClient( "mongodb://192.168.1.110:27017/" )
    mydb = myClient['Details'] # The strategy orders will be stored in database named MockStrategySignal 
    coll = mydb['expiries']

    temp = coll.find_one()
    current_exp = datetime.strptime(temp["current"], "%Y-%m-%d")

    next_exp = datetime.strptime(temp["next"], "%Y-%m-%d")
    far_exp = datetime.strptime(temp["far"], "%Y-%m-%d")

    s_current= current_exp.strftime("%Y-%b-%d").upper()
    s_next = next_exp.strftime("%Y-%b-%d").upper()
    s_far = far_exp.strftime("%Y-%b-%d").upper()

    current_mon = s_current[5:8]
    next_mon = s_next[5:8]
    far_mon = s_far[5:8]

    current_expiry_symbol = ''
    next_expiry_symbol = ''
    names_with_expiry = []

    #print("s_current",s_current)
   # print("s_next", s_next) 

    if(instrument=="OPT"):

        if(current_mon==next_mon):
            if(current_mon=="JAN"):
                current_expiry_symbol = symbol + s_current[2:4] +"1" +s_current[-2:]

            elif(current_mon=="FEB"):
                current_expiry_symbol = symbol + s_current[2:4] +"2" +s_current[-2:]

            elif(current_mon=="MAR"):
                current_expiry_symbol = symbol + s_current[2:4] +"3" +s_current[-2:]

            elif(current_mon=="APR"):
                current_expiry_symbol = symbol + s_current[2:4] +"4" +s_current[-2:]

            elif(current_mon=="MAY"):
                current_expiry_symbol = symbol + s_current[2:4] +"5" +s_current[-2:]

            elif(current_mon=="JUN"):
                current_expiry_symbol = symbol + s_current[2:4] +"6" +s_current[-2:]

            else:
                current_expiry_symbol = symbol + s_current[2:4] +current_mon[0] +s_current[-2:]
            

        else:
            current_expiry_symbol = symbol + s_current[2:4] +current_mon


        if(next_mon==far_mon):

            if(next_mon=="JAN"):
                next_expiry_symbol = symbol + s_next[2:4] +"1" +s_next[-2:]

            elif(next_mon=="FEB"):
                next_expiry_symbol = symbol + s_next[2:4] +"2" +s_next[-2:]

            elif(next_mon=="MAR"):
                next_expiry_symbol = symbol + s_next[2:4] +"3" +s_next[-2:]
            elif(next_mon=="APR"):
                next_expiry_symbol = symbol + s_next[2:4] +"4" +s_next[-2:]

            elif(next_mon=="MAY"):
                next_expiry_symbol = symbol + s_next[2:4] +"5" +s_next[-2:]

            elif(next_mon=="JUN"):
                next_expiry_symbol = symbol + s_next[2:4] +"6" +s_next[-2:]

            else:
                next_expiry_symbol = symbol + s_next[2:4] + next_mon[0] +s_next[-2:]
            

        else:
            next_expiry_symbol = symbol + s_next[2:4] +next_mon

    


        

        
        names_with_expiry = [current_expiry_symbol, next_expiry_symbol]  
        

    # For old data format

    if(instrument=="FUT"):
        current_expiry_symbol = symbol + s_current[2:4] +current_mon + "FUT"
        names_with_expiry = [current_expiry_symbol]


    return(names_with_expiry)


########################################################################################################





t=1
while t:
    
    #sys.path.insert(1, 'C:/Users/admin/BPS')
    #from AttachExpiry import get_expiry_names
    current_expiry_date=get_expiry_names("BANKNIFTY","OPT")[0]
    far_expiry_date=get_expiry_names("BANKNIFTY","OPT")[1]

    RedisClientInd = redis.Redis(host='192.168.1.110', port='6379', db='7',decode_responses=True)
    #RedisClientOpt = redis.Redis(host='192.168.1.110', port='6379', db='6',decode_responses=True)
    sym_name = "BankNifty"
    index_data=json.loads(RedisClientInd.get(sym_name))
    price_index_live = float(index_data['Touchline']['LastTradedPrice'])
    
    RedisClientInd_Fut = redis.Redis(host='192.168.1.110', port='6379', db='5',decode_responses=True)
    #RedisClientOpt = redis.Redis(host='192.168.1.110', port='6379', db='6',decode_responses=True)
    sym_name_Fut = "BANKNIFTY23JUNFUT"
    index_data_Fut=json.loads(RedisClientInd_Fut.get(sym_name_Fut))
    price_index_live_Fut = float(index_data_Fut['Touchline']['LastTradedPrice'])  
    
    
    #read index spot values from data
    index_spot=price_index_live
    
    #rounding off index spot
    rem=index_spot%100
    if rem>=50:
        strike_spot=int(index_spot/100)*100+100
    else:
        strike_spot=int(index_spot/100)*100
    
    strike_spot-=1000 

    
    index_futures=price_index_live_Fut
    
    dict={
        "index spot":index_spot,
        "index futures":index_futures,
        "strikes":{
       
            }
    }

    s=dict['strikes']
    
    i=0
    while i<21:
    #set a variable to fetch near and far expiry value for the corresponding strike
        s[strike_spot]=[]
        #creating new symbol name to fetch expiry values
        add_strike=str(strike_spot)
        new_sym_name_current=str(current_expiry_date)+add_strike+"CE"
        new_sym_name_far=str(far_expiry_date)+add_strike+"CE"
        
        RedisClientInd_curr = redis.Redis(host='192.168.1.110', port='6379', db='6',decode_responses=True)
        #RedisClientOpt = redis.Redis(host='192.168.1.110', port='6379', db='6',decode_responses=True)
        index_data_curr=json.loads(RedisClientInd_curr.get(new_sym_name_current))
        curr_expiryvalue = float(index_data_curr['Touchline']['LastTradedPrice'])   
        
        RedisClientInd_far = redis.Redis(host='192.168.1.110', port='6379', db='6',decode_responses=True)
        #RedisClientOpt = redis.Redis(host='192.168.1.110', port='6379', db='6',decode_responses=True)
        index_data_far=json.loads(RedisClientInd_far.get(new_sym_name_far))
        far_expiryvalue = float(index_data_far['Touchline']['LastTradedPrice'])         
        
        s[strike_spot].append(curr_expiryvalue)
        s[strike_spot].append(far_expiryvalue)
        
        strike_spot+=100

        i+=1

    with open("sample.json","w")as outfile:
        json.dump(dict,outfile)

   # print(dict)


    ws=xw.Book('SamplePrint.xlsx').sheets("Sheet1")

    #importing json file
    with open('sample.json','r')as openfile:
        json_object=json.load(openfile)
    index_spot=json_object.get('index spot')
    index_futures=json_object.get('index futures')

    #placing value in excel
    ws.range("A1").value='BANKNIFTY'
    ws.range("A2").value=index_spot

    ws.range("B1").value='BANKNIFTYFUTURES'
    ws.range("B2").value=index_futures

    ws.range("A5").value='far expiry'
    ws.range("B5").value='current expiry'
    ws.range("C5").value='STRIKES'

    strikerates=json_object.get('strikes')
    rownum=6
    for keys,values in strikerates.items():
        ws.range('C'+str(rownum)).value=keys
        ws.range('B'+str(rownum)).value=values[0]
        ws.range('A'+str(rownum)).value=values[1]
        rownum+=1
   
    #outputting the table
    #print("Data :\n",ws.range("A1").expand().value)

    sleep(1)







#3.printing data in excel



KeyboardInterrupt: 